In [1]:
import time
import pandas as pd
pd.set_option('display.max_columns', 500)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# path to webdriver for selenium
service = Service(r"..\browser\geckodriver.exe")

driver = webdriver.Firefox(service=service)
waitTime = 10

site = "https://www.rotowire.com/basketball/rotations.php"
driver.get(site)
time.sleep(5)

teams = ["ATL", "BOS", "BKN", "CHA", "CHI", "CLE", "DAL", "DEN", "DET", "GSW",
         "HOU","IND","LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NOP", "NYK", 
         "OKC", "ORL", "PHI", "PHX", "POR", "SAC", "SAS", "TOR", "UTA", "WAS"]

player_info = {}
for i in teams:
    htmlID = "rotation-table-{}".format(i)
    team_data = driver.find_element(By.ID, htmlID)
    # retrieve its page coords
    loc = team_data.location
    #scroll to the location to activate the table creation
    driver.execute_script("window.scrollTo(0, {});".format(str(loc["y"])))
    # pause for data loading
    time.sleep(2)
    # grabbing all of the player elements for a team
    players = team_data.find_elements(By.TAG_NAME, "a")
    for p in players:
        # grabbing the name and href to extract the rotoID
        player_name = p.text
        href= p.get_attribute("href")
        rotoId = int(href.split("-")[-1])
        player_info[player_name] = rotoId

driver.close()

#convert dict to dataframe
data = pd.DataFrame.from_dict(player_info, orient='index', columns=['rotowireId']).reset_index()
data.rename(columns={"index":"player"}, inplace=True)

#export
data.to_csv("../data/rotoId.csv", index=False)